## Automating Spreadsheet Data for Reporting

### Methodology & Working Assumptions

Following acronyms under TRANTYPE are as such:<br>

CANC - Cancellation (assumed to be the cancelled GWP)<br>
REIN – Reinstatement (assumed to be part of booked GWP)<br>
RNWL – Renewal (assumed to be part of booked GWP)<br>
ENDO - Endowment (assumed to be part of booked GWP)<br>
NWBS - Not guessable (assumed to be part of booked GWP)<br>

<ul>
    <li>Cancelled Gross Written Premium assumed to refer to the net sum under CANC in TRANTYPE</li>
    <li>Booked Gross Written Premium assumed to refer to the net sum under REIN + RNWL + ENDO + NWBS in TRANTYPE</li>
    <li>The definition of showing figures up to the current month refers to the most recent transaction date in "YrM" column of the most recent file run through this script.</li>
</ul>
<br>
Premium sheet is the main file of reference but since LOB information is in Policy Sheet, it will be joined via common policy number "CHDRNUM" so we can provide aggregates by LOB as well. The main columns to examine would be "YrM" (reference date for the years and months breakdown analysis), "TRANTYPE" (to determine whether policy was booked or cancelled) and GWP (premium amount).

#### Import Libraries

In [1]:
import pandas as pd
from datetime import datetime

#### Read the Excel Document

In [2]:
df_policy= pd.read_excel (r'PREMIUM_POLICY_DATA.xlsx', sheet_name='POLICY')
df_premium= pd.read_excel (r'PREMIUM_POLICY_DATA.xlsx', sheet_name='PREMIUM')

#### Verify each spreadsheet has been extracted successfully

In [3]:
df_policy.head()

,CHDRNUM,lob,CNTTYPE,FUNDCODE,ACSTYP,D_latestexpiry
0,10201150,017-Travel,ATP,L,NO,2021-09-30
1,10201150,017-Travel,ATP,L,NO,2021-09-30
2,10201150,017-Travel,ATP,L,NO,2021-09-30
3,10201150,017-Travel,ATP,L,NO,2021-09-30
4,10201150,017-Travel,ATP,L,NO,2021-09-30


In [4]:
df_premium.head()

,CHDRNUM,TRANNO,AGENTID,YrM,D_tran,D_eff,D_com,D_exp,TRANTYPE,FINANCIAL,GWP
0,10201150,1897,86,201912,2019-12-20,2019-11-03,2019-10-01,2020-09-30,ENDO,YES,0.0
1,10201150,1898,86,201912,2019-12-26,2019-12-15,2019-10-01,2020-09-30,ENDO,YES,149.9
2,10201150,1899,86,201912,2019-12-28,2019-10-12,2019-10-01,2020-09-30,ENDO,YES,87.1
3,10201150,1900,86,201912,2019-12-28,2019-10-12,2019-10-01,2020-09-30,ENDO,YES,-89.7
4,10201150,1901,86,201912,2019-12-28,2019-09-29,2018-10-01,2019-09-30,ENDO,YES,30.0


Check variable data types

In [5]:
df_policy.dtypes

CHDRNUM                   object
lob                       object
CNTTYPE                   object
FUNDCODE                  object
ACSTYP                    object
D_latestexpiry    datetime64[ns]
dtype: object

In [6]:
df_premium.dtypes

CHDRNUM              object
TRANNO                int64
AGENTID               int64
YrM                   int64
D_tran       datetime64[ns]
D_eff        datetime64[ns]
D_com        datetime64[ns]
D_exp        datetime64[ns]
TRANTYPE             object
FINANCIAL            object
GWP                 float64
dtype: object

In [7]:
# Inspect for completeness

print('Policy Sheet contains {0} rows and {1} columns.'.format(df_policy.shape[0],df_policy.shape[1]))
print('Premium Sheet contains {0} rows and {1} columns.'.format(df_premium.shape[0],df_premium.shape[1]))

Policy Sheet contains 28398 rows and 6 columns.
Premium Sheet contains 28398 rows and 11 columns.


In [8]:
# Check for any null fields

print(df_policy.isnull().sum())

CHDRNUM           0
lob               0
CNTTYPE           0
FUNDCODE          0
ACSTYP            0
D_latestexpiry    0
dtype: int64


In [9]:
print(df_premium.isnull().sum())

CHDRNUM      0
TRANNO       0
AGENTID      0
YrM          0
D_tran       0
D_eff        0
D_com        0
D_exp        0
TRANTYPE     0
FINANCIAL    0
GWP          0
dtype: int64


All rows are present with no null values.

#### Merge into one dataframe

In [10]:
# Drop duplicates from policy sheet which might cause excess joining rows

df_policy.drop_duplicates(subset='CHDRNUM', inplace=True)

In [11]:
df_combined = pd.merge(df_premium, df_policy, on='CHDRNUM', how='left')

In [12]:
df_combined.head(5)

,CHDRNUM,TRANNO,AGENTID,YrM,D_tran,D_eff,D_com,D_exp,TRANTYPE,FINANCIAL,GWP,lob,CNTTYPE,FUNDCODE,ACSTYP,D_latestexpiry
0,10201150,1897,86,201912,2019-12-20,2019-11-03,2019-10-01,2020-09-30,ENDO,YES,0.0,017-Travel,ATP,L,NO,2021-09-30
1,10201150,1898,86,201912,2019-12-26,2019-12-15,2019-10-01,2020-09-30,ENDO,YES,149.9,017-Travel,ATP,L,NO,2021-09-30
2,10201150,1899,86,201912,2019-12-28,2019-10-12,2019-10-01,2020-09-30,ENDO,YES,87.1,017-Travel,ATP,L,NO,2021-09-30
3,10201150,1900,86,201912,2019-12-28,2019-10-12,2019-10-01,2020-09-30,ENDO,YES,-89.7,017-Travel,ATP,L,NO,2021-09-30
4,10201150,1901,86,201912,2019-12-28,2019-09-29,2018-10-01,2019-09-30,ENDO,YES,30.0,017-Travel,ATP,L,NO,2021-09-30


In [13]:
df_combined.shape

(28398, 16)

### Filter only values relevant to current month and year and previous year period

Since the only relevant values are Year-To-Date months and also the similar period for previous year, let us first extract only these information before doing the further data sorting. We will base this on time period on the "YrM" column.There are 2 approaches that can be used here. 
<ul>
    <li>One would be the use of datetime format range manipulation.</li> 
<li>The other would be date integer format to work around. This will be selected for now</li> 
    </ul>

In [14]:
# Set the anchor points for all the period ranges

most_recent_date = df_combined['YrM'].nlargest().iloc[0]
current_month = str(most_recent_date)[4:]
current_year = str(most_recent_date)[:4]
previous_year = int(current_year) - 1

In [15]:
if current_month != '01':
    current_date_range = [int(current_year + '01'), most_recent_date]
    previous_date_range = [int(str(previous_year) + '01'), int(str(previous_year) + current_month)]
else:
    current_date_range = [most_recent_date, most_recent_date]
    previous_date_range = [int(str(previous_year) + '01'), int(str(previous_year) + '01')]

df_current_year = df_combined[(df_combined['YrM'] >= current_date_range[0]) & (df_combined['YrM'] <= current_date_range[1])]

df_previous_year = df_combined[(df_combined['YrM'] >= previous_date_range[0]) & (df_combined['YrM'] <= previous_date_range[1])]

df_final = pd.concat([df_current_year, df_previous_year])

#### Format YrM to be more readable with new name and date formats

In [16]:
df_final.rename(columns={'YrM':'Year_Month'}, inplace=True)
df_final['Year_Month']= pd.to_datetime(df_final['Year_Month'], format="%Y%m").dt.strftime('%Y-%m')

In [17]:
df_final.shape

(27206, 16)

### Generate Breakdown of Gross Written Premium Cancelled

In [18]:
df_cancel = df_final.loc[df_final['TRANTYPE'] == "CANC"]

In [19]:
# Filter via groupby

df_gwp_cancel = df_cancel.groupby(["lob","Year_Month"])["GWP"].sum().to_frame()
df_gwp_cancel

GWP
lob                   Year_Month          
017-Travel            2020-01    -14202.92
                      2020-02     -6550.37
018-Personal Accident 2020-01     -6873.85
                      2020-02     -7997.22

Allocation seems to be correct and matching the spreadsheet.

In [20]:
# Convert the table into a full dataframe

df_cancelled = df_gwp_cancel.reset_index(level=['lob', 'Year_Month'])
df_cancelled

,lob,Year_Month,GWP
0,017-Travel,2020-01,-14202.92
1,017-Travel,2020-02,-6550.37
2,018-Personal Accident,2020-01,-6873.85
3,018-Personal Accident,2020-02,-7997.22


### Generate Breakdown of Gross Written Premium Booked

In [21]:
df_booked = df_final.loc[df_combined['TRANTYPE'] != "CANC"]

In [22]:
df_gwp_booked = df_booked.groupby(["lob","Year_Month"])["GWP"].sum().to_frame()

In [23]:
# Convert the table into a full dataframe

df_booked = df_gwp_booked.reset_index(level=['lob', 'Year_Month'])
df_booked

,lob,Year_Month,GWP
0,017-Travel,2020-01,715553.64
1,017-Travel,2020-02,457142.34
2,018-Personal Accident,2020-01,622758.09
3,018-Personal Accident,2020-02,312239.69


#### Verify both the Cancelled & Booked amounts tally with original dataset

In [24]:
print(df_gwp_cancel['GWP'].sum())
print(df_gwp_booked['GWP'].sum())

-35624.36000000001
2107693.759999979


In [25]:
df_final.groupby(df_combined['TRANTYPE'] == 'CANC')['GWP'].sum()

TRANTYPE
False    2107693.76
True      -35624.36
Name: GWP, dtype: float64

Both the amounts match correctly.

### Construct the Report

In [26]:
df_cancelled.rename(columns = {"GWP":"GWP Cancelled"}, inplace = True)
df_booked.rename(columns = {"GWP":"GWP Booked"}, inplace = True)

In [27]:
df_cancelled['GWP Booked'] = df_booked["GWP Booked"]

In [28]:
df_export = df_cancelled
df_export

,lob,Year_Month,GWP Cancelled,GWP Booked
0,017-Travel,2020-01,-14202.92,715553.64
1,017-Travel,2020-02,-6550.37,457142.34
2,018-Personal Accident,2020-01,-6873.85,622758.09
3,018-Personal Accident,2020-02,-7997.22,312239.69


#### Export the Report

In [29]:
df_export.to_excel("AXA_Monthly_Report.xlsx",index = False)  